# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.768463  ,  2.3276033 , -3.7457774 , -1.8541754 , -5.169583  ,
         7.667088  ,  2.7118318 ,  8.495609  ,  1.7038484 ,  1.1884269 ],
       [ 4.6476874 ,  8.37788   , -9.070581  ,  9.459332  ,  8.450423  ,
        -1.0210547 ,  3.3920872 , -7.8629856 , -0.75276613,  0.48384127],
       [-2.9414442 ,  4.6401706 , -4.5027533 ,  2.2855108 ,  1.644645  ,
        -2.4937892 , -5.2241607 , -1.5499196 , -8.063638  ,  2.8169358 ],
       [-4.271077  ,  5.561165  , -5.6640925 , -1.8229512 , -9.2925    ,
         0.730283  ,  4.4586773 , -2.8876226 , -5.125775  ,  9.694357  ],
       [ 5.5837426 , -4.1515303 ,  4.369667  , -3.0020502 ,  3.6388965 ,
        -4.341912  , -3.3187115 ,  6.503671  , -6.865036  , -1.0266498 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:44048 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 251.80 GiB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.8098736 ,  8.422671  , -9.239024  ,  9.379146  ,  8.499883  ,
        -1.0592818 ,  3.3437867 , -7.802613  , -0.5946331 ,  0.26447597,
         5.507395  , -4.1069803 ,  4.2890773 , -2.817205  ,  3.615015  ,
        -4.1612997 , -3.6209638 ,  6.2185287 , -6.946047  , -1.0828305 ,
        -5.8267717 ,  2.225876  , -3.8601224 , -1.6974077 , -5.313416  ,
         7.579578  ,  2.9187477 ,  8.540423  ,  1.5523204 ,  1.0841813 ],
       [-2.8941853 ,  4.4741883 , -4.447566  ,  2.3820994 ,  1.7478832 ,
        -2.5046248 , -5.2083306 , -1.6937685 , -8.134754  ,  2.6468306 ,
        -4.3163633 ,  5.5655394 , -5.732199  , -1.7384956 , -9.344656  ,
         0.70846593,  4.435839  , -2.9008994 , -4.948641  ,  9.695302  ,
         8.366521  , -6.2474537 , -6.3494735 ,  1.9546977 ,  4.1576157 ,
        -9.167906  ,  4.6070685 ,  8.788585  ,  6.8644238 ,  2.2319877 ],
       [-4.6657143 , -9.558953  ,  6.6572285 ,  4.4401307 ,  2.1730297 ,
         2.5904047 ,  0.58001   ,  6.255036  , -8